In [19]:
import plotly
import plotly.plotly as py
import pandas as pd

plotly.tools.set_credentials_file(username='dev10110', api_key='0sQ9Wb2f7HVlEVE3eRal')

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df.head()

df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [10]:
df.head()

,name,pop,lat,lon,text
0,New York,8287238,40.730599,-73.986581,New York <br>Population 8.287238 million
1,Los Angeles,3826423,34.053717,-118.242727,Los Angeles <br>Population 3.826423 million
2,Chicago,2705627,41.875555,-87.624421,Chicago <br>Population 2.705627 million
3,Houston,2129784,29.758938,-95.367697,Houston <br>Population 2.129784 million
4,Philadelphia,1539313,39.952335,-75.163789,Philadelphia <br>Population 1.539313 million


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3228 entries, 0 to 3227
Data columns (total 5 columns):
name    3228 non-null object
pop     3228 non-null int64
lat     3228 non-null float64
lon     3228 non-null float64
text    3228 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 126.2+ KB


In [211]:
import csv
import geocoder
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import statistics
import scipy.optimize as scipyopt
import pandas as pd
import math
import requests
import json




## Set up plotly
plotly.tools.set_credentials_file(username='dev10110', api_key='0sQ9Wb2f7HVlEVE3eRal')

mapbox_access_token = 'pk.eyJ1IjoiZGV2MTAxMTAiLCJhIjoiY2ptNGp0bTYyMHE1azNqb3R3eXU2bG1mdiJ9.XOyqUI4vtvW4ZqQxnbl6Lg'

## import data 

#(survivorlatdata,survivorlondata,survivorpresent,survivorabsent)=tag_photos()

(vollatdata,vollondata)=volunteerLoc(['K1MQ','KB1IXO-9','EW3846'])


## import data 
#import volunteer data and plot raw scatter
#vollatdata=[]
#vollondata=[]
#with open('volunteerLocData.csv',newline='') as csvfile:
#    spamreader = csv.reader(csvfile)
#    for row in spamreader:
#        vollatdata.append(float(row[0]))
#        vollondata.append(float(row[1]))
    #latlon=[[float(row[0]), float(row[1])] for row in spamreader]

    
# create the volunteers plot
voltrace=dict(
    type = 'scattermapbox',
    lat = vollatdata, lon = vollondata,
    mode='markers',
    #marker = dict(),
    name = 'Volunteer')


 #get survivors location
if 1:
    survivorlatdata=[]
    survivorlondata=[]
    survivorpresent=[]
    survivorabsent =[]
    with open('survivorLocData.csv',newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            survivorlatdata.append(float(row[0]))
            survivorlondata.append(float(row[1]))
            survivorpresent.append(float(row[2]))
            survivorabsent.append(float(row[3]))

# mark the survivor locations
survivortrace=go.Scattermapbox(
    lat = survivorlatdata, 
    lon = survivorlondata,
    #size = [10*surv for surv in survivorpresent],
    mode='markers',
    name='Survivor!',
    marker = dict(
        size=10,
        color='#ff0000')
)


#get users location
g = geocoder.ip('me')
mylat=g.latlng[0]
mylon=g.latlng[1]

# mark the user location
mytrace=dict(
    type = 'scattermapbox',
    lat = [mylat], lon = [mylon],
    mode='markers',
    marker = dict(
    symbol='star',
    color='#003bff'),
    name='Map generated here'
    )





## create the mapping

#create a cost matrix


#heuristic1: distance between nodes squared
costmatrix=np.zeros((len(vollatdata), len(survivorlatdata)))

for i in range(0,len(vollatdata)):
    for j in range(0,len(survivorlatdata)):
        costmatrix[i,j]=distance(vollatdata[i],vollondata[i],survivorlatdata[j],survivorlondata[j]) #compute the dist between volunteer and survivor
        #ideally should include some confidence info/weather info

bestassignmentrow, bestassignmentcol=scipyopt.linear_sum_assignment(costmatrix)

#create the first assignment
#assignments = go.scattergeo(
 #   lat = [vollatdata[bestassignmentrow[0]],survivorlatdata[bestassignmentcol[0]]], 
  #  lon = [vollondata[bestassignmentrow[0]],survivorlondata[bestassignmentcol[0]]], 
    #size = [10*surv for surv in survivorpresent],
   # )

assignments = []
for i in range(0, len( vollatdata ) ):
    assignments.append(
        dict(
            type = 'scattergeo',
            lat = [vollatdata[bestassignmentrow[i]], survivorlatdata[bestassignmentcol[i]]],
            lon = [vollondata[bestassignmentrow[i]], survivorlondata[bestassignmentcol[i]]],
            mode = 'lines',
            line = dict(
                width = 10,
                color = 'red',
            )
        )
    )





##make the plot look nice

medianlat=statistics.median(survivorlatdata)
medianlon=statistics.median(survivorlondata)

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=medianlat,
            lon=medianlon
        ),
        pitch=0,
        zoom=6,
        style='light'
    ),
)
data=[voltrace, mytrace,survivortrace]+assignments
## finally implement the plot
fig=dict(data=[voltrace, mytrace,survivortrace]+assignments,layout=layout)

py.iplot(fig,filename='Where help is needed')










In [164]:
import scipy.optimize as scipyopt
import scipy.optimize as scipyopt

#create a cost matrix
#heuristic1: distance between nodes squared

costmatrix=np.zeros((len(vollatdata), len(survivorlatdata)))

for i in range(0,len(vollatdata)):
    for j in range(0,len(survivorlatdata)):
        costmatrix[i,j]=distance(vollatdata[i],vollondata[i],survivorlatdata[j],survivorlondata[j])


bestassignmentrow, bestassignmentcol=scipyopt.linear_sum_assignment(costmatrix)

In [165]:
bestassignmentrow

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [133]:
import math
def distance(lat, lng, lat0, lng0):
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0)*math.cos(lat0)
    return deglen*math.sqrt(x*x + y*y)

In [170]:
bestassignmentcol[0]

15

In [173]:
s=[1,2]
b=[3,4]
s+b

[1, 2, 3, 4]

In [172]:
s

[1, 2, [3, 4]]

In [186]:
from watson_developer_cloud import VisualRecognitionV3
import json
import os
import getLatLong as geotag

#all photos must be stored in 'Photos' directory (same dir as code)

#tagged_images = [] #[(lat,long),roof_true probability, roof_false probability]
lats = []
longs = []
rt = []
rf = []

visual_recognition = VisualRecognitionV3(
    '2018-03-19',
    iam_apikey='vFTSKJUVMqQs9h4mHY0wwqI9yPjYS6z1tbv_ZkCfV3SX')

def tag_photos():
    
    for file in os.listdir('Photos'):

        direc = 'Photos/' + file

        with open(direc, 'rb') as images_file:
            classes = visual_recognition.classify(
                images_file,
                threshold='0.0',
                owners=["me"]).get_result()
                #classifier_ids=['default']).get_result()

            roof_false = classes['images'][0]['classifiers'][0]['classes'][0]['score']
            roof_true = classes['images'][0]['classifiers'][0]['classes'][1]['score']


        (lat,long) = geotag.getLatLong(direc)

        lats.append(lat)
        longs.append(long)
        rt.append(roof_true)
        rf.append(roof_false)
        #tagged_images.append([(lat,long),roof_true,roof_false])
        
    return(lats,longs,rt,rf)

print(tag_photos())



([-24.500008746954016, 0.2323], [33.35785150844174, 0.2323], [0.035, 0.044], [0.853, 0.135])


In [188]:


#### extra functions
def distance(lat, lng, lat0, lng0):
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0)*math.cos(lat0)
    return deglen*math.sqrt(x*x + y*y)
    

In [191]:
[vollatdata[bestassignmentrow[0]],survivorlatdata[bestassignmentcol[0]]]

[43.53761829, -24.500008746954016]

In [197]:
i=1
[vollatdata[bestassignmentrow[i]], survivorlatdata[bestassignmentcol[i]]]

[43.24938912, -24.500008746954016]

In [203]:
[voltrace, mytrace,survivortrace]+assignments

[{'type': 'scattermapbox',
  'lat': [43.53761829,
   43.24938912,
   43.5909836,
   43.93571255,
   43.68341471,
   43.98766855,
   43.65638631,
   43.36782512,
   43.33337972,
   43.3005574,
   43.29989354,
   43.98079066,
   43.93793936,
   43.98924527,
   43.91646716],
  'lon': [-72.27733466,
   -72.81933067,
   -72.20935957,
   -72.74645273,
   -72.52164719,
   -72.56784585,
   -72.08093735,
   -72.92360347,
   -72.22616272,
   -72.44851428,
   -72.42631115,
   -72.78298457,
   -72.01686595,
   -72.98209167,
   -72.123395],
  'mode': 'markers',
  'name': 'Volunteer'},
 {'type': 'scattermapbox',
  'lat': [42.3646],
  'lon': [-71.1028],
  'mode': 'markers',
  'marker': {'symbol': 'star', 'color': '#003bff'},
  'name': 'Map generated here'},
 Scattermapbox({
     'lat': [-24.500008746954016, 0.2323, -24.500008746954016, 0.2323,
             -24.500008746954016, 0.2323, -24.500008746954016, 0.2323,
             -24.500008746954016, 0.2323, -24.500008746954016, 0.2323,
             -24.

In [210]:

aprs = []
locData = {}
#input list of volunteer's callsigns, returns

def volunteerLoc(callList):

    aprsWeb = 'https://api.aprs.fi/api/get?name='
    aprsWeb+=callList[0]
    
    for i in range(1,len(callList)):

        aprsWeb+=','
        call = callList[i]
        aprsWeb+= call 


    aprsWeb+='&what=loc&apikey=116932.M4GQQFbrZzOx9&format=json'
    grabaprs = requests.get(aprsWeb)
    jlist = grabaprs.content
    aprs = json.loads(jlist)

    for i in range(0,len(callList)):
        
        locData[aprs['entries'][i]['name']] = [aprs['entries'][i]['lat'],aprs['entries'][i]['lng']]
        
    vollat = [float(location[0]) for location in locData.values()]
    vollong = [float(location[1]) for location in locData.values()]

    return vollat,vollong
    

print(volunteerLoc(['K1MQ','KB1IXO-9','EW3846']))



([41.645, 41.72699, 41.70083], [-72.708, -72.58122, -73.03517])
